Topic of Notebook
============

<span style="color:Coral;">This notebook is used to determine the performance of the algorithms. The following parameters are measured: Duration of execution, size of the roadmap, success rate. The results are saved in an Excel file after execution.</span>


This is interactive notebook regarding "Introduction to path planning". (Author: Björn Hein)

License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)


## Adding all planners
<!--  -->


In [11]:
import sys
from HelperPackage import HelperClass
sys.path.append("templates")
%load_ext autoreload
%autoreload 2
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from HelperPackage.IPPerfMonitor import IPPerfMonitor

from BasicPRM import IPVISBasicPRM 
from BasicPRM import IPBasicPRM_Roundtrip

from VisibilityPRM import IPVisibilityPRM_Roundtrip
from VisibilityPRM import IPVISVisibilityPRM

from VisibilityPRM import CC_IPVisibilityPRM_Roundtrip

from LazyPRM import IPLazyPRM_Roundtrip
from LazyPRM import IPVISLazyPRM

from IPython.utils import io



## Set-up of the test scenario and the configuration for all planner


Following a procedure to compare all discussed planners are shown:

1. Configuration for every planner is defined
2. The configuration and the planner is stored in the variable setup, a Python dict()
3. the variable setup is then used to uniformly call the planning calls


In [13]:
plannerFactory = dict()

basicConfig = dict()
basicConfig["radius"] = 3
basicConfig["numNodes"] = 200
plannerFactory["basePRM"] = [IPBasicPRM_Roundtrip.BasicPRM, basicConfig, IPVISBasicPRM.basicPRMVisualize]


visbilityConfig = dict()
visbilityConfig["ntry"] = 300
plannerFactory["visibilityPRM"] = [IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]


visbilityConfig = dict()
visbilityConfig["ntry"] = 300
plannerFactory["visibilityPRMOptimized"] = [CC_IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]


lazyConfig = dict()
lazyConfig["initialRoadmapSize"] = 10
lazyConfig["updateRoadmapSize"]  = 5
lazyConfig["kNearest"] = 8
plannerFactory["lazyPRM"] = [IPLazyPRM_Roundtrip.LazyPRM, lazyConfig, IPVISLazyPRM.lazyPRMVisualize]

In [14]:
class ResultCollection (object):
    
    def __init__(self, plannerFactoryName, planner, benchmark, solution, perfDataFrame):
        self.plannerFactoryName = plannerFactoryName
        self.planner = planner
        self.benchmark = benchmark
        self.solution = solution
        self.perfDataFrame = perfDataFrame

In [15]:
from HelperPackage import IPTestSuite
import importlib

In [16]:
importlib.reload(IPTestSuite)

<module 'HelperPackage.IPTestSuite' from 'c:\\Users\\andre\\OneDrive - Hochschule Karlsruhe\\HS Karlsruhe\\1. Semester\\Roboterprogrammierung\\Projekt\\Git\\RKIM23RobotProgramDS\\02_eigeneNotebooks\\HelperPackage\\IPTestSuite.py'>

In [17]:
fullBenchList = IPTestSuite.benchList

for benchmark in fullBenchList:
    print(benchmark.name)

Trap
Bottleneck
Fat bottleneck
B_rush
Circle of Death


In [18]:
sum_time_Basic = [[],[],[]]
sum_time_Visibility = [[],[],[]]
sum_time_Visibility_Optimized= [[],[],[]]
sum_time_Lazy = [[],[],[]]

list_of_benchmarks = []

In [19]:
resultList = list()
testList = fullBenchList[:5]

amountOfRepetition = 0
i = 0
while (i< amountOfRepetition):
    testList.extend(fullBenchList[:5])
    i = i + 1
 
print(testList)

for key,producer in list(plannerFactory.items()):
    HelperClass.HelperClass.printInColor(str(key)+" "+ str(producer), 'yellow')

    for benchmark in testList:
        planner = producer[0](benchmark.collisionChecker)
        IPPerfMonitor.clearData()

        try:

            print(benchmark.name)

            with io.capture_output() as captured:
                start = time.time()
                resultList.append(ResultCollection(key,
                                            planner, 
                                            benchmark, 
                                            planner.planRoundPath(benchmark.startList,benchmark.interimGoalList,producer[1]),
                                            IPPerfMonitor.dataFrame()),)
                end = time.time()
                duration = end - start
            if (key == "basePRM"):
                sum_time_Basic[0].append(duration)
                                   
                sum_time_Basic[1].append(len(resultList[0].planner.graph.nodes()))
                if (resultList[0].solution != []):
                    sum_time_Basic[2].append(1)
                else:
                    sum_time_Basic[2].append(0)

                
            elif (key == "visibilityPRM"):
                sum_time_Visibility[0].append(duration)
                sum_time_Visibility[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Visibility[2].append(1)
                else:
                    sum_time_Visibility[2].append(0)
            
            elif (key == "visibilityPRMOptimized"):
                sum_time_Visibility_Optimized[0].append(duration)
                
                sum_time_Visibility_Optimized[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Visibility_Optimized[2].append(1)
                else:
                    sum_time_Visibility_Optimized[2].append(0)


            
            elif (key == "lazyPRM"):
                sum_time_Lazy[0].append(duration)
                sum_time_Lazy[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Lazy[2].append(1)
                else:
                    sum_time_Lazy[2].append(0)

            list_of_benchmarks.append(benchmark.name)

            resultList = list()

        except Exception as e:
            HelperClass.HelperClass.printInColor("PLANNING ERROR ! PLANNING ERROR ! PLANNING ERROR", 'red')
            HelperClass.HelperClass.printInColor(e, 'red')
            import traceback
            HelperClass.HelperClass.printInColor(traceback.print_exc(), 'red')

            pass

        

[<HelperPackage.IPBenchmark.Benchmark object at 0x0000027EBA1F08D0>, <HelperPackage.IPBenchmark.Benchmark object at 0x0000027EB9D13990>, <HelperPackage.IPBenchmark.Benchmark object at 0x0000027EB9D13A10>, <HelperPackage.IPBenchmark.Benchmark object at 0x0000027EB9D13F90>, <HelperPackage.IPBenchmark.Benchmark object at 0x0000027EB9D12010>]


Trap
Bottleneck
Fat bottleneck
B_rush
Circle of Death


Trap
Bottleneck
Fat bottleneck
B_rush
Circle of Death


Trap
Bottleneck
Fat bottleneck
B_rush
Circle of Death


Trap
Bottleneck
Fat bottleneck
B_rush
Circle of Death


In [21]:
print(sum_time_Lazy)
import pandas as pd
import os
from openpyxl import load_workbook
import time

result_frames = []

print(len(sum_time_Basic[0]))

for i in range(len(sum_time_Basic[0])):
    
    # Create DataFrame
    data = {
        "Benchmark": [list_of_benchmarks[i]],
        "Basic Zeit": [sum_time_Basic[0][i]],
        "Basic Nodes": [sum_time_Basic[1][i]],
        "Basic Erfolgreich": [sum_time_Basic[2][i]],
        "Visibility Zeit ": [sum_time_Visibility[0][i]],
        "Visibility Nodes ": [sum_time_Visibility[1][i]],
        "Visibility Erfolgreich ": [sum_time_Visibility[2][i]],
        "Optimized_Visibility Zeit ": [sum_time_Visibility_Optimized[0][i]],
        "Optimized_Visibility Nodes ": [sum_time_Visibility_Optimized[1][i]],
        "Optimized_Visibility Erfolgreich ": [sum_time_Visibility_Optimized[2][i]],
        "LazyPRM Zeit ": [sum_time_Lazy[0][i]],
        "LazyPRM Nodes ": [sum_time_Lazy[1][i]],
        "LazyPRM Erfolgreich ": [sum_time_Lazy[2][i]],
    }

    df = pd.DataFrame(data)

    # Append DataFrame to list
    result_frames.append(df)

final_df = pd.concat(result_frames, ignore_index=True)
print(type(final_df))

# Write DataFrame to Excel
final_df.to_excel("Auswertung_" + str(time.time()) + ".xlsx", index=False)


[[0.013514041900634766, 0.20586013793945312, 0.04803466796875, 0.730454683303833, 0.24076390266418457], [48, 502, 147, 445, 491], [1, 0, 1, 1, 0]]
5
<class 'pandas.core.frame.DataFrame'>
